### Add Callbacks + Metrics
Adding more setup, by-epoch, teardown methods to learner

Using new callback `TestSetRecorder` - we run validate on a third dataset. (I still need to figure out if tfms's are being applied to this third dataset...it should if test_accuracy matches my_test_metrics on an accurate model)

Using `subcat_color_acc`, `subcat_piece_acc` we add metrics to be monitored at each epoch, for correctly predicting a piece's player-color and piece-class individually. What could this show?

In [1]:
from fastai2.vision.all import *
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

from modules.trainutils import (build_dl, 
                                piece_class_parse,
                                my_metrics, 
                                my_test_metrics,
                                stratify_sample)

from modules.trainutils import (subcat_color_acc,
                                subcat_piece_acc)

from modules.learnutils import (TestSetRecorder,
                                get_cb, 
                                learner_add_testset
                               )

test_path = Path('../../../other-chess-data/regulation-test-all/')
train_path = Path('../../../rf-chess-data/cropped_v1/')

##### Build Data

In [2]:
test_dl = build_dl(test_path, n=None)
len(test_dl.train.items)

228

In [3]:
n = 200
seed=42

train_dl = ImageDataLoaders.from_name_func(
                train_path, 
                # get_image_files(train_path),
                stratify_sample(train_path, n=n, np_seed=seed),
                valid_pct=0.2, 
                seed=42,
                label_func=piece_class_parse, 
                item_tfms=RandomResizedCrop(128, min_scale=0.5),
                batch_tfms=aug_transforms(),
)

##### Setup Custom Learner

In [4]:
learn = cnn_learner(train_dl, resnet18, 
                    metrics=[accuracy, 
                             subcat_color_acc, 
                             subcat_piece_acc,
                            ]
                   )

In [5]:
learner_add_testset(learn, test_dl)

In [6]:
[len(learn.dls[i].items) for i in (0,1,2)]

[154, 38, 228]

In [8]:
learn.add_cb(TestSetRecorder())